# Exemplary analysis of the PBMC3K single-cell RNA dataset

In [ ]:
import csv
import os
import math
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import mygene

from anndata import AnnData
import scanpy as sc

# keggtool imports
from keggtools import (
    Pathway,
    Enrichment,
    EnrichmentResult,
    Resolver,
    Storage,
    Renderer,
    IMMUNE_SYSTEM_PATHWAYS,
)

# Used folders
rawdata_dir = os.path.join(os.getcwd(), "rawdata")
figure_dir = os.path.join(os.getcwd(), "figures")

# Global settings
sc.settings.verbosity = 0
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

# Ignore all warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Analysis of PBMC3K

Analysis base on `scanpy` tutorial ([https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html)).


### Preprocessing of dataset

In [ ]:
# Load count data
adata: AnnData = sc.read_10x_mtx(rawdata_dir, var_names="gene_symbols", cache=False)
adata.var_names_make_unique()

In [ ]:
# Filter genes and cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
# Calculate percentage of mitochodrial genes in each cell
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

In [ ]:
# Plot QC parameter
sc.pl.violin(adata, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], jitter=0.4, multi_panel=True)

In [ ]:
# Filter for QC parameter
adata = adata[adata.obs["n_genes_by_counts"] < 2500, :]
adata = adata[adata.obs["pct_counts_mt"] < 5, :]

In [ ]:
# Normalize counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
# Isolate highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata.raw = adata
adata = adata[:, adata.var["highly_variable"]]

In [ ]:
# Regress out counts and mitochondrial genes and scale dataset
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])
sc.pp.scale(adata, max_value=10)

### PCA analysis and clustering

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata, flavor="igraph", n_iterations=2)

In [ ]:
# TODO: plot marker genes of cluster
sc.pl.umap(adata, color=["CST3", "NKG7"])

In [ ]:
sc.pl.violin(adata, ["CST3", "NKG7", "PPBP"], groupby="leiden", rotation=90.0)

In [ ]:
# Rename leiden clusters
new_cluster_names: list[str] = [
    "CD4+ T-cells",
    "CD14+ Monocytes",
    "B-cells",
    "CD8+ T-cells",
    "NK cells",
    "FCGR3A+ Monocytes",
    "Dendritic cells",
    "Megakaryocytes",
]
adata.rename_categories("leiden", new_cluster_names)

In [ ]:
# Plot and save UMAP
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.set_aspect("equal", adjustable="box")

sc.pl.umap(adata, color="leiden", legend_loc="right margin", title="", frameon=True, ax=ax, show=False)

fig.tight_layout(pad=3.0)
fig.savefig(os.path.join(figure_dir, "figure2.png"))

### Differential analysis results to pandas dataframe

In [ ]:
sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon", n_genes=100)

In [ ]:
# Export diff exp data
diffexp_df = sc.get.rank_genes_groups_df(adata, group="NK cells", pval_cutoff=0.05)
diffexp_df.head()

### Perform KEGGTOOLS enrichment analysis

In [ ]:
organism_id: str = "hsa"

In [ ]:
from keggtools.utils import merge_entrez_geneid

In [ ]:
diffexp_df = merge_entrez_geneid(diffexp_df)
diffexp_df.head()

In [ ]:
# Init resolver instance
resolver: Resolver = Resolver()
pathway_list: list[Pathway] = []

# Download all immune system pathways
for number, _ in IMMUNE_SYSTEM_PATHWAYS.items():
    pathway_list.append(resolver.get_pathway(organism=organism_id, code=number))

In [ ]:
# Init enrichment from list of immune system pathways
analysis: Enrichment = Enrichment(pathways=pathway_list)
analysis_result: list[EnrichmentResult] = analysis.run_analysis(gene_list=list(diffexp_df["entrez"]))
result_df: pd.DataFrame = analysis.to_dataframe()

# Filter out pathways with no genes found
result_df = result_df[result_df["study_count"] > 0]

# Print out result
result_df.head()

### Plot results of enrichment analysis

In [ ]:
plt.figure(figsize=(8, 5), dpi=300)
scatter = plt.scatter(
    x=result_df["study_count"] / result_df["pathway_genes"] * 100,
    y=result_df["pathway_title"],
    c=[-math.log10(x) for x in result_df["pvalue"]],
    cmap="coolwarm",
)

cbar = plt.colorbar()
cbar.set_label("- log10(p value)")

plt.grid(visible=None)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "figure4.png"), bbox_inches="tight")
plt.show()

### Plot pathway

* "Antigen processing and presentation" (hsa:04612) show a significant p value

In [ ]:
pathway: Pathway = resolver.get_pathway(organism=organism_id, code="04612")

In [ ]:
# diffexp_df[["entrez"]] = diffexp_df[["entrez"]].astype(int)
overlay: dict = dict(zip(list(diffexp_df["entrez"]), list(diffexp_df["logfoldchanges"]), strict=True))

In [ ]:
renderer: Renderer = Renderer(kegg_pathway=pathway, gene_dict=overlay, cache_or_resolver=resolver.storage)
renderer.render(display_unlabeled_genes=True)

In [ ]:
# Save dot string to file
with open(os.path.join(figure_dir, "figure5.dot"), "w") as file_obj:
    file_obj.write(renderer.to_string())

# Save binary data to file
renderer.to_file(filename=os.path.join(figure_dir, "figure5.png"), extension="png")

In [ ]:
# Display image
from IPython.display import Image, display

img: Image = Image(os.path.join(figure_dir, "figure5.png"))
display(img)